# Parallelization Lab
In this lab, you will be leveraging several concepts you have learned to obtain a list of links from a web page and crawl and index the pages referenced by those links - both sequentially and in parallel. Follow the steps below to complete the lab.

### Step 1: Use the requests library to retrieve the content from the URL below.

In [1]:
import requests

url = 'https://en.wikipedia.org/wiki/Data_science'

In [2]:
response = requests.get(url)

### Step 2: Use BeautifulSoup to extract a list of all the unique links on the page.

In [3]:
from bs4 import BeautifulSoup, SoupStrainer

In [4]:
soup = BeautifulSoup(response.text, 'html.parser')
# print(soup)
res = soup.find_all("a", href=True)
listoflinks = []
for el in res:
    listoflinks.append(el['href'])
listoflinks

['#mw-head',
 '#searchInput',
 '/wiki/Information_science',
 '/wiki/File:PIA23792-1600x1200(1).jpg',
 '/wiki/File:PIA23792-1600x1200(1).jpg',
 '/wiki/Comet_NEOWISE',
 '/wiki/Astronomical_survey',
 '/wiki/Space_telescope',
 '/wiki/Wide-field_Infrared_Survey_Explorer',
 '/wiki/Interdisciplinary',
 '/wiki/Scientific_method',
 '/wiki/Algorithm',
 '/wiki/Knowledge',
 '/wiki/Unstructured_data',
 '#cite_note-1',
 '#cite_note-2',
 '/wiki/Data_mining',
 '/wiki/Machine_learning',
 '/wiki/Big_data',
 '/wiki/Computational_statistics',
 '/wiki/Analytics',
 '#cite_note-:2-3',
 '/wiki/Statistics',
 '/wiki/Data_analysis',
 '/wiki/Informatics',
 '/wiki/Scientific_method',
 '/wiki/Phenomena',
 '/wiki/Data',
 '#cite_note-4',
 '/wiki/Mathematics',
 '/wiki/Computer_science',
 '/wiki/Information_science',
 '/wiki/Domain_knowledge',
 '#cite_note-:2-3',
 '/wiki/Computer_science',
 '/wiki/Turing_Award',
 '/wiki/Jim_Gray_(computer_scientist)',
 '/wiki/Empirical_research',
 '/wiki/Basic_research',
 '/wiki/Comput

### Step 3: Use list comprehensions with conditions to clean the link list.
There are two types of links, absolute and relative. Absolute links have the full URL and begin with http while relative links begin with a forward slash (/) and point to an internal page within the wikipedia.org domain. Clean the respective types of URLs as follows.

Absolute Links: Create a list of these and remove any that contain a percentage sign (%).

Relative Links: Create a list of these, add the domain to the link so that you have the full URL, and remove any that contain a percentage sign (%).

Combine the list of absolute and relative links and ensure there are no duplicate

In [5]:
domain = 'http://wikipedia.org'

In [6]:
absolutelinks = [link for link in listoflinks if ('%' not in link) and (link.startswith("http"))]

In [7]:
relativelinks = [domain + link for link in listoflinks if ('%' not in link) and not (link.startswith("//")) and (link.startswith('/'))]

In [8]:
fulllist = list(set(absolutelinks + relativelinks))

In [9]:
fulllist

['https://magazine.amstat.org/blog/2015/10/01/asa-statement-on-the-role-of-statistics-in-data-science/',
 'https://foundation.wikimedia.org/wiki/Privacy_policy',
 'http://wikipedia.org/wiki/Information_technology',
 'http://wikipedia.org/wiki/New_York_City',
 'http://wikipedia.org/wiki/Data_philanthropy',
 'http://wikipedia.org/wiki/Andrew_Gelman',
 'http://wikipedia.org/wiki/Wikipedia:Community_portal',
 'http://wikipedia.org/wiki/Montpellier_2_University',
 'http://wikipedia.org/wiki/Graphic_design',
 'http://wikipedia.org/wiki/Computer_science',
 'http://wikipedia.org/wiki/ISBN_(identifier)',
 'http://wikipedia.org/w/index.php?title=Data_science&action=edit&section=4',
 'http://wikipedia.org/w/index.php?title=Special:UserLogin&returnto=Data+science',
 'http://wikipedia.org/w/index.php?title=Data_science&action=history',
 'http://wikipedia.org/wiki/Data_degradation',
 'http://wikipedia.org/wiki/Extract,_load,_transform',
 'http://wikipedia.org/wiki/Data_scraping',
 'http://wikipedia.

### Step 4: Use the os library to create a folder called wikipedia and make that the current working directory.

In [10]:
import os

In [11]:
newpath = r'/Users/tinym/Documents/IronHack/DAFT_1022/Module_1/Lab_34_Parallelization/wikipedia'
if not os.path.exists(newpath):
    os.makedirs(newpath)
os.chdir(newpath)

In [12]:
os.getcwd()

'/Users/tinym/Documents/IronHack/DAFT_1022/Module_1/Lab_34_Parallelization/wikipedia'

### Step 5: Write a function called index_page that accepts a link and does the following.
Tries to request the content of the page referenced by that link.

Slugifies the filename using the slugify function from the python-slugify library and adds a .html file extension.

      If you don't already have the python-slugify library installed, you can pip install it as follows:
      $ pip3 install python-slugify.

      To import the slugify function, you would do the following: from slugify import slugify.

      You can then slugify a link as follows slugify(link).

Creates a file in the wikipedia folder using the slugified filename and writes the contents of the page to the file.

If an exception occurs during the process above, just pass.

In [13]:
from slugify import slugify

In [20]:
def index_page(link):
    try:
        response = requests.get(link)
        soup = BeautifulSoup(response.text, 'html.parser')
        filename = slugify(link).lower() + ".html"
        os.makedirs(os.path.dirname(filename), exist_ok=True)
        with open('./wikipedia' + '/' + filename, "wb") as f:
            f.write(soup)
    except:
        pass

### Step 6: Sequentially loop through the list of links, running the index_page function each time.
Remember to include %%time at the beginning of the cell so that it measures the time it takes for the cell to run.

In [21]:
%%time

for link in fulllist:
    index_page(link)

CPU times: user 27.6 s, sys: 974 ms, total: 28.6 s
Wall time: 1min 52s


### Step 7: Perform the page indexing in parallel and note the difference in performance.
Remember to include %%time at the beginning of the cell so that it measures the time it takes for the cell to run.

In [15]:
import multiprocessing 

In [19]:
%%time
plist = []
for link in fulllist:
    process = multiprocessing.Process(target=index_page, args=(link,))
    process.start()
    plist.append(process)

for process in plist:
    process.join()

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/tinym/opt/anaconda3/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/tinym/opt/anaconda3/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Users/tinym/opt/anaconda3/lib/python3.9/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'index_page' on <module '__main__' (built-in)>
    exitcode = _main(fd, parent_sentinel)
  File "/Users/tinym/opt/anaconda3/lib/python3.9/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'index_page' on <module '__main__' (built-in)>
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/tinym/opt/anaconda3/lib/python3

CPU times: user 708 ms, sys: 2.59 s, total: 3.3 s
Wall time: 4.19 s


Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/tinym/opt/anaconda3/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Users/tinym/opt/anaconda3/lib/python3.9/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'index_page' on <module '__main__' (built-in)>
